In [5]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['문서 요약']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess



#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            transcript = line['transcript']
            highlights = line['highlight']
            for highlight in highlights:
                highlight_start = highlight['start_time']
                highlight_end = highlight['end_time']
                ## add highlight to transcript in start and end time
                highlight['transcript'] = []
                for transcript_line in transcript:
                    if highlight_start <= transcript_line['end'] and transcript_line['start'] <= highlight_end:
                        highlight['transcript'].append(transcript_line)
            #### data preprocess end 
                #### 문서 요약
                data = {'input': None, 'output': None}
                ## preprocess data from line
                data['input'] = '\n'.join([line['form'] for line in highlight['transcript']])
                data['output'] = highlight['summary']
                ## preprocess data from line end
                task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
                #### 문서 요약 end
            
            
            
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

train:   0%|          | 0/8 [00:00<?, ?it/s]

valid: 100%|██████████| 9/9 [00:00<00:00, 20.92it/s]
